In [2]:
import requests
import csv
import psycopg2

# Function to handle conversion of data types
def convert_to_int(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return None

# Fetch data from API
url = 'https://api.sampleapis.com/baseball/hitsSingleSeason'
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    try:
        players = response.json()
        if isinstance(players, list) and len(players) > 0:
            # Print the structure of the first player to understand the columns
            print("Player data structure:", players[0])

            # Save raw data to CSV file
            try:
                with open('players_data_raw.csv', 'w', newline='', encoding='utf-8') as csvfile:
                    fieldnames = players[0].keys()
                    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

                    writer.writeheader()
                    for player in players:
                        writer.writerow(player)
                print("Raw player data saved to players_data_raw.csv")
            except Exception as e:
                print(f"Error saving raw data to CSV: {e}")

            # Save processed data to CSV file
            try:
                with open('players_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
                    fieldnames = ['id', 'rank', 'player', 'age_that_year', 'hits', 'year', 'bats']
                    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

                    writer.writeheader()
                    for player in players:
                        writer.writerow({
                            'id': convert_to_int(player.get('id')),
                            'rank': convert_to_int(player.get('Rank')),
                            'player': player.get('Player'),
                            'age_that_year': convert_to_int(player.get('AgeThatYear')),
                            'hits': convert_to_int(player.get('Hits')),
                            'year': convert_to_int(player.get('Year')),
                            'bats': player.get('Bats')
                        })
                print("Processed player data saved to players_data.csv")
            except Exception as e:
                print(f"Error saving processed data to CSV: {e}")

        else:
            print("No player data found.")
    except requests.exceptions.JSONDecodeError:
        print("Error decoding JSON from response.")
        players = []
else:
    print(f"Error fetching data: {response.status_code}")
    players = []

# Insert data into database only if players is a list of dictionaries
if isinstance(players, list):
    conn = psycopg2.connect(
        dbname="baseball",
        user="postgres",
        password="!Expr3sso",
        host="127.0.0.1"
    )
    cur = conn.cursor()
    for player in players:
        # Ensure each player is a dictionary
        if isinstance(player, dict):
            cur.execute("""
                INSERT INTO players (id, rank, player, age_that_year, hits, year, bats)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            """, (
                convert_to_int(player.get('id')),
                convert_to_int(player.get('Rank')),
                player.get('Player'),
                convert_to_int(player.get('AgeThatYear')),
                convert_to_int(player.get('Hits')),
                convert_to_int(player.get('Year')),
                player.get('Bats')
            ))
    conn.commit()
    cur.close()
    conn.close()
else:
    print("The API response is not a list of players.")


Player data structure: {'Rank': '1', 'Player': 'Ichiro\xa0Suzuki', 'AgeThatYear': '30', 'Hits': 262, 'Year': 2004, 'Bats': 'L', 'id': 1}
Raw player data saved to players_data_raw.csv
Processed player data saved to players_data.csv


UniqueViolation: duplicate key value violates unique constraint "players_pkey"
DETAIL:  Key (id)=(1) already exists.
